<a href="https://colab.research.google.com/github/kuuuun/python_jupyter_notes/blob/main/chapter07_StatAggr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# chapter07 统计与聚合

In [3]:
import numpy as np

## 简单的求和与平均

In [4]:
# 生成一个模拟数据：3个班级，每个班级 4 名学生的考试成绩
rng = np.random.default_rng(42)
scores = rng.integers(60, 100, size=(3, 4))

print("原始成绩单:\n", scores)

# 1. 全局聚合
# 不指定 axis，就把所有数字当成一大袋子数
total_score = np.sum(scores)
avg_score = np.mean(scores)

print(f"\n所有分数的总和: {total_score}")
print(f"全校平均分: {avg_score:.2f}")

原始成绩单:
 [[63 90 86 77]
 [77 94 63 87]
 [68 63 81 99]]

所有分数的总和: 948
全校平均分: 79.00


## 7.1.2 核心难点：理解统计中的 Axis

当我们在聚合函数中指定 axis 时，请记住这个心法：

“指定哪个轴，就是要把哪个轴压扁。”

axis=0 （垂直轴）：把行压扁 -> 结果代表 每一列 的统计值。

axis=1 （水平轴）：把列压扁 -> 结果代表 每一行 的统计值。

In [5]:
# scores 形状是 (3, 4) -> (班级, 学生)

# 场景 A：我想看每个班级的平均分
# 思考：我要在这个计算中保留“班级”维度，消灭“学生”维度。
# “学生”在 axis=1，所以我们要压扁 axis=1
class_avgs = scores.mean(axis=1)
print(f"各班平均分: {class_avgs}")
# 结果形状 (3,)，对应 3 个班级

# 场景 B：我想看每一门课（假设列代表不同科目，虽不太符合生成逻辑，但为了演示）的最高分
# 思考：我要消灭“班级”维度，保留“科目”维度。
# “班级”在 axis=0，所以压扁 axis=0
subject_max = scores.max(axis=0)
print(f"各科目最高分: {subject_max}")
# 结果形状 (4,)，对应 4 个学生位置

各班平均分: [79.   80.25 77.75]
各科目最高分: [77 94 86 99]


> **🌟 小贴士** ：  
> 别去背“0是列，1是行”。要看数据的物理意义。如果数据是 `(Time, City, Sensor)` ，你想算出每个城市所有时间的平均值，那就是要把 `Time` 和 `Sensor` 对应的轴都消灭掉： `data.mean(axis=(0, 2))` 。是的， `axis` 可以是一个元组！

## 救命参数：keepdims=True

In [6]:
# 不使用 keepdims
means = scores.mean(axis=1)
print(f"普通均值形状: {means.shape}") # (3,)

# 使用 keepdims
means_kept = scores.mean(axis=1, keepdims=True)
print(f"保持维度形状: {means_kept.shape}") # (3, 1)

# 现在可以安全地进行去中心化操作
centered_scores = scores - means_kept
print("\n去中心化后的数据片段:\n", centered_scores[:2, :])

普通均值形状: (3,)
保持维度形状: (3, 1)

去中心化后的数据片段:
 [[-16.    11.     7.    -2.  ]
 [ -3.25  13.75 -17.25   6.75]]


## 病毒传播

In [7]:
dirty_data = np.array([1.0, 2.0, np.nan, 4.0])

print("普通 sum:", np.sum(dirty_data))  # 输出: nan
print("普通 mean:", np.mean(dirty_data)) # 输出: nan

普通 sum: nan
普通 mean: nan


## 疫苗：nan 安全函数

NumPy 提供了一整套以 `nan` 开头的函数，它们会自动 **忽略** `NaN` 值进行统计。

| 普通函数 | 安全函数 | 作用 |
| --- | --- | --- |
| `np.sum` | **`np.nansum`** | 求和，把 nan 当作 0 |
| `np.mean` | **`np.nanmean`** | 求平均，分母会减去 nan 的数量 |
| `np.max` | **`np.nanmax`** | 求最大值 |
| `np.std` | **`np.nanstd`** | 求标准差 |

In [8]:
print("安全 sum:", np.nansum(dirty_data))  # 1+2+4 = 7.0
print("安全 mean:", np.nanmean(dirty_data)) # 7.0 / 3 = 2.33 (分母是3不是4)

安全 sum: 7.0
安全 mean: 2.3333333333333335


In [9]:
import numpy as np

# 定义两个矩阵
A = np.array([[1, 2],
              [3, 4]])
B = np.array([[5, 6],
              [7, 8]])

print("矩阵 A:\n", A)
print("矩阵 B:\n", B)

# 1. 逐元素乘法 (*)
# 对应位置相乘：1*5, 2*6, ...
print("\n逐元素乘法 (A * B):\n", A * B)

# 2. 矩阵乘法 (@)
# 行乘列：1*5+2*7, 1*6+2*8, ...
print("\n矩阵乘法 (A @ B):\n", A @ B)

# 3. 向量的点积
v1 = np.array([1, 2, 3])
v2 = np.array([4, 5, 6])
# 1*4 + 2*5 + 3*6 = 32
print(f"\n向量内积 (v1 @ v2): {v1 @ v2}")

矩阵 A:
 [[1 2]
 [3 4]]
矩阵 B:
 [[5 6]
 [7 8]]

逐元素乘法 (A * B):
 [[ 5 12]
 [21 32]]

矩阵乘法 (A @ B):
 [[19 22]
 [43 50]]

向量内积 (v1 @ v2): 32
